# Deploy a Model

After the experiments on your data have been run and a suitable model is selected, the next step is to create an inference Skill that uses our model. 

Please install `cortex-python`,`cortex-python[builders]` for builder functionality, `cortex-python[viz]` for vizualizations. 


## Select the Model

In this example, the first cell runs a setup notebook similar to the [Experiment annotations notebook](https://docs.cortex.insights.ai/docs/cortex-python-sdk-guide/experiment-annotations/). Then, the example imports the last run from an experiment ran on the [UCI Iris Dataset](https://archive.ics.uci.edu/ml/datasets/Iris). 

In [ ]:
%run ./00-setup.ipynb

In [ ]:
from cortex import Cortex, Message

client = Cortex.client()
exp = client.experiment('flowers/experiment')

exp

After running the last cell, the experiment displays runs from the experiments notebook. Select the last of these runs and use that as the model for the skill.

In [ ]:
run = exp.last_run() 

model = run.get_artifact('model')

print('Run id: %s \nModel name: "%s"' % (run.id, model.name))

## Create an action using the model

A skill's action builds from the model with a builder supplied by the Cortex Client. In the following example, the action name is `flowers/iris-predict`. 

A model may depend on other python libraries. To make those libraries available to the action, specify them in the `with_requirements` method. Only specify libraries that your model needs that are not provided by the Cortex Python SDK[ * ](#requirements). The following example requires the keras library and the TensorFlow backend. The final `build` command builds a docker image and deploys an Action to run the image when invoked.

In [ ]:
builder = client.builder()
builder.action('flowers/iris-predict')\
        .with_requirements(['tensorflow>=1.12,<2',
                            'keras>=2.2.4,<3'])\
        .from_model(model)\
        .build()

Instantiate the newly saved action to be tested.

In [ ]:
action = client.action('flowers/iris-predict')
action

Test the action by invoking it to ensures the action is ready for use.

In [ ]:
params = {'columns': ['sepal_len','sepal_w','petal_len','petal_w'],
           'values': [[4.9,3.1,1.5,0.2]]}

result = action.invoke(message=client.message(params))
print(result.payload)

In the following cell, the maximum value for the three-node output of the neural network selects and matches itself to the appropriate type of iris flower. 

In [ ]:
x = numpy.argmax(result.payload['values'][0])

iris_dict = {0:'Iris-setosa',1:'Iris-versicolor',2:'Iris-virginica'}

iris_dict[x]

## Create a skill using the action

To create a skill from the action, call the `builder` to get a skill builder. 

In [ ]:
skill_builder = builder.skill('flowers/iris-predict')\
     .title('Iris Flower Prediction')\
     .description('Measurement based prediction for the type of Iris flowers.')


A resulting schema is used by the skill builder to identify elements of the input. In this example, the schema describes the measurement values for parts of the Iris flower. 

In [ ]:
schema_name = 'flowers/iris-input-schema'

builder.schema(schema_name)\
     .title('Iris Prediction Schema Instance')\
     .from_parameters([{'name': 'values', 'type': 'array', 'format': 'float32'}]).build()

skill_builder = skill_builder.input('iris-measurement').title('Iris Measurements')\
     .use_schema(schema_name)\
     .all_routing(action, 'measurement-based-prediction').build()

Specify the skill construction. The construction describes the skill including a name that matches the action name and a description of the output.

In [ ]:
skill_builder = skill_builder.output('measurement-based-prediction').title('Classification')\
    .parameter(name='IrisClass', type='array', format='float32').build()

With the skill `builder` fully specified, call `to_camel` so the skill's contents can be reviewed.

In [ ]:
skill_builder.to_camel()

## Publish the skill

Invoke the builder to publish the skill. Upon publication, the skill can be used in Cortex Studio with other skills to compose agents. 

In [ ]:
skill = skill_builder.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

## Requirements

The Cortex Python SDK provides the following requirements and do not need to be added to an action's Docker Image. 

* seaborn>=0.9.0,<0.10
* dill==0.2.8.2
* discovery-transitioning-utils>=1.3.50,<2
* fdk==0.0.31
* dataclasses>=0.6
* seaborn>=0.9.0,<0.10
* matplotlib>=2.2.2,<3
* more_itertools>=4.3.0,<5
* pyyaml>=3.13,<4
* cuid>=0.3,<1
* maya==0.5.0
* docker==3.5.0
* deprecation==2.0.6
* tenacity==5.0.2